# Asynch adata converter to msg 

In [14]:
%matplotlib inline 
from ifis_tools import asynch_manager as am 
from ifis_tools import database_tools as db
import pandas as pd
import numpy as np
import glob
from multiprocessing import Pool

In [2]:
USGS_links = db.SQL_USGS_at_IFIS()

In [8]:
L = glob.glob('/media/nicolas/Data/RunoffExp/Raw/190_*.dat')
L.sort()

In [13]:
L[0].split('_')[1][:-4]

'2008'

In [15]:
LinksID = []
for k in USGS_links.keys():
    LinksID.append(USGS_links[k])

In [21]:
def warp(link):
    try:
        Lista = []
        for ano in L:
            Lista.append(Data[str(link)+'_'+ano.split('_')[1]])
        D = pd.concat(Lista)
        D2 = pd.Series(np.zeros(dates.size), dates)
        D2[D.index] = D.values
        D2[D2 == 0] = np.percentile(D.values,10)
        D2.to_msgpack('/media/nicolas/Data/RunoffExp/Results/HLM254/'+str(link)+'_'+model+'.msg')
    except:
        pass
    
def warp190(link):
    try:
        Lista = []
        for ano in L:
            Lista.append(Data[str(link)+'_'+ano.split('_')[1][:-4]])
        D = pd.concat(Lista)
        D2 = pd.Series(np.zeros(dates.size), dates)
        D2[D.index] = D.values
        D2[D2 == 0] = np.percentile(D.values,10)
        D2.to_msgpack('/media/nicolas/Data/RunoffExp/Results/HLM190/'+str(link)+'.msg')
    except:
        pass

## Process data for asynch model 

In [ ]:
dates = pd.date_range('2008-04-01','2018-12-30', freq = '15min')

for model in ['01',]:#'009','007','005','003']:
    
    L = glob.glob('/media/nicolas/Data/RunoffExp/Raw/254_*'+model+'.dat')
    L.sort()
    
    for p in [[0,87],[87,173]]:
        
        Data = {}
        for l in L:
            #try:
            d = am.ASYNCH_results(l)
            for link in LinksID[p[0]:p[1]]:
                try:
                    Data.update({str(link)+'_'+l.split('_')[1] : d.ASYNCH_dat2Serie(link, l.split('_')[1]+'-04-01 00:00', freq = '15min')})
                except:
                    pass
            print(l.split('_')[1])

        Worker = Pool(processes=7)
        Worker.map(warp, LinksID[p[0]:p[1]])
        Worker.close()
        Worker.join()
    
    print(model)

## Process data for asynch 190

In [25]:
L = glob.glob('/media/nicolas/Data/RunoffExp/Raw/190_*.dat')
L.sort()

for p in [[0,87],[87,173]]:

    Data = {}
    for l in L:
        #try:
        d = am.ASYNCH_results(l)
        for link in LinksID[p[0]:p[1]]:
            try:
                Data.update({str(link)+'_'+l.split('_')[1][:-4] : d.ASYNCH_dat2Serie(link, l.split('_')[1][:-4]+'-04-01 00:00', freq = '15min')})
            except:
                pass
    
    Worker = Pool(processes=7)
    Worker.map(warp190, LinksID[p[0]:p[1]])
    Worker.close()
    Worker.join()
    
    print(p)

[0, 87]
[87, 173]


In [26]:
L = glob.glob('/media/nicolas/Data/RunoffExp/Results/HLM190/*')
print(len(L))

165


# Mean rainfall for each link

In [25]:
def warp(link):
    R =  db.SQL_Get_MeanRainfall(link, '2008-04-01','2018-12-1')
    R.to_msgpack('/media/nicolas/Data/RunoffExp/Results/Rainfall/Rain_'+str(link)+'.msg')

In [11]:
%%time
p = Pool(processes=7)
p.map(warp, LinksID[14:])
p.close()
p.join()


CPU times: user 961 ms, sys: 482 ms, total: 1.44 s
Wall time: 18min 15s
